## Exporting a ResNet50 model to ONNX
You can export any Fireball model to ONNX using the ```exportToOnnx``` function. This notebook shows how to use this function to create an ONNX model. It assumes that a trained ResNet50 model already exists in the ```Models``` directory. Please refer to the notebook [Image Classification with ResNet50](ResNet50.ipynb) for more info about using a pretrained ResNet50 model.

Fireball can also export models with reduced number of parameters, pruned models, and quatized models. Please refer to the following notebooks for more information:

- [Reducing number of parameters of ResNet50 Model](ResNet50-Reduce.ipynb)
- [Pruning ResNet50 Model](ResNet50-Prune.ipynb)
- [Quantizing ResNet50 Model](ResNet50-Quantize.ipynb)

Note: Fireball uses the ```onnx``` python package to export models to ONNX. We also use the ```onnxruntime``` here to run and evaluate the onnx models. If these packages are not installed already, you can just run the following commands in a new cell and restart the kernel.
```
%pip install onnx==1.7.0
%pip install onnxruntime==1.5.2
```

## Load a pretrained model

In [1]:
from fireball import Model
from fireball.datasets.imagenet import ImageNetDSet
gpus='0,1,2,3'

# Create the test dataset for evaluation.
testDs = ImageNetDSet.makeDatasets('Test', batchSize=256, preProcessing='Crop256Cafe', numWorkers=8)

# orgFileName = "Models/ResNet50.fbm"        # Original model
# orgFileName = "Models/ResNet50QR.fbm"      # Quantized - Retrained
# orgFileName = "Models/ResNet50PR.fbm"      # Pruned - Retrained
# orgFileName = "Models/ResNet50PRQR.fbm"    # Pruned - Retrained - Quantized - Retrained
# orgFileName = "Models/ResNet50RR.fbm"      # Reduced - Retrained
# orgFileName = "Models/ResNet50RRQR.fbm"    # Reduced - Retrained - Quantized - Retrained
# orgFileName = "Models/ResNet50RRPR.fbm"    # Reduced - Retrained - Pruned - Retrained
orgFileName = "Models/ResNet50RRPRQR.fbm"  # Reduced - Retrained - Pruned - Retrained - Quantized - Retrained

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus=gpus)
model.printLayersInfo()
model.initSession()
results = model.evaluate()


Reading from "Models/ResNet50RRPRQR.fbm" ... Done.
Creating the fireball model "ResNet50" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 7 2 3               112 112 64    None                      7,119      
S1_L2_BN         112 112 64                             56 56 64      ReLU     MP(KSP):3 2 1    256        
S2_L1_RES2       56 56 64      2 Paths, 8 layers        56 56 256     ReLU                      48,621     
S2_L2_RES1       56 56 256     2 Paths, 6 layers        56 56 256     ReLU                      48,346     
S2_L3_RES1       56 56 256     2 Paths, 6 layers        56 56 256     ReLU                      48,863     
S3_L1_RES2c1     56 56 256     2 P

## Export the model and check the exported ONNX model

In [2]:
onnxFileName = orgFileName.replace(".fbm",".onnx")

model.exportToOnnx(onnxFileName, runQuantized=True, classNames=ImageNetDSet.classNames, 
                   modelDocStr="Fireball example: ResNet50 Model")

# Check the exported model. This throws exceptions if something is wrong with the exported model.
import onnx
from onnx import shape_inference

onnxModel = onnx.load(onnxFileName)
onnx.checker.check_model(onnxModel)


Exporting to ONNX model "Models/ResNet50RRPRQR.onnx" ... 
    Processed all 21 layers.                                      
    Saving to "Models/ResNet50RRPRQR.onnx" ... Done.
Done (55.66 Sec.)


## Using netron to visualize the exported model
We can now visualize the model's network structure using the "netron" package. If netron is not installed, you can just run the following command in a new cell and restart the kernel.

```
%pip install netron
```

In [3]:
import netron
import platform

if platform.system() == 'Darwin':      # Running on MAC
    netron.start(onnxFileName)   
else:
    import socket
    hostIp = socket.gethostbyname(socket.gethostname())
    netron.start(onnxFileName, address=(hostIp,8084))

Serving 'Models/ResNet50RRPRQR.onnx' at http://10.21.16.50:8084


## Running inference on the exported model
To verify the exported model, we can now run inference on it. Here we load an image and do the required pre-processing before passing it to the exported model as input. We then print the top-3 most probable predicted labels for the image.

In [4]:
import cv2
import numpy as np
imageFileName = 'CoffeeMug.jpg'

img = cv2.imread(imageFileName)     # Reads image in BGR order

# Resize the image to 256x256
imgSize = img.shape[:2]
ratio = 256.0/min(imgSize)
newSize = (int(np.round(imgSize[1]*ratio)), int(np.round(imgSize[0]*ratio)))

# Note: INTER_AREA is best when shrinking and CV_INTER_CUBIC is best when enlarging
img = cv2.resize(img, newSize,  interpolation = (cv2.INTER_AREA if ratio<1.0 else cv2.INTER_CUBIC))

# Now crop the center 224x224 image
dw = newSize[0] - 224
dh = newSize[1] - 224
resizedImg = img[dh//2:dh//2+224, dw//2:dw//2+224,:]

# Normalize the image using the mean values for blue, green, and red
inputImage = (np.float32(resizedImg) - [103.939, 116.779, 123.68])
inputImage = np.transpose(inputImage, (2,0,1))    # Onnx expects channel-first images

# Inference using the ONNX model and "onnxruntime"
import onnxruntime as ort
session = ort.InferenceSession(onnxModel.SerializeToString(), None)
    
print('input: %s, ouput: %s'%(session.get_inputs()[0].name,session.get_outputs()[0].name))
y = session.run(['ClassProbs'],{'InputImage':[inputImage]})

classProbs = y[0][0]
top3Indexes = np.argsort(classProbs)[-3:][::-1]    # Indexes of classes with 3 highest probs (decreasing order)
top3Porbs = classProbs[top3Indexes]
print('Top-3 Classes (For "%s"):'%(imageFileName))
for i in range(3):
    print('    %s (%f)'%(ImageNetDSet.classNames[top3Indexes[i]], top3Porbs[i])) 

input: InputImage, ouput: ClassProbs
Top-3 Classes (For "CoffeeMug.jpg"):
    coffee_mug (0.778762)
    cup (0.121574)
    pitcher (0.047829)


## Where do I go from here?

[Exporting ResNet50 Model to CoreML](ResNet50-CoreML.ipynb)

[Exporting ResNet50 Model to TensorFlow](ResNet50-TF.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Image Classification with ResNet50](ResNet50.ipynb)

[Reducing number of parameters of ResNet50 Model](ResNet50-Reduce.ipynb)

[Pruning ResNet50 Model](ResNet50-Prune.ipynb)

[Quantizing ResNet50 Model](ResNet50-Quantize.ipynb)
